In [1]:
from config.simpleconv_config import SimpleConvConfig
from models.simpleconv import SimpleConv
from studies.study_factory import StudyFactory
from utils.pre_processor import PreProcessor
from utils.fetch import fetch_audio_and_brain_pairs

import torch
device = 'cuda'

In [ ]:
brain_sample_rate = 100
frequency_bands = {"all": (0.5, 100)}
subject, task, session = 0, 3, 0
seed = 42
max_random_shift = 1
window_size = 4
n_jobs = -1

study = StudyFactory().create_study("gwilliams", path="data/gwilliams")
pre_processor = PreProcessor(
    brain_sample_rate=brain_sample_rate,
)

In [3]:
brain_segments, audio_segments, layout = fetch_audio_and_brain_pairs(
    subject=subject,
    task=task,
    session=session,
    max_random_shift=max_random_shift,
    window_size=window_size,
    study=study,
    pre_processor=pre_processor,
    frequency_bands=frequency_bands,
    audio_sample_rate=16000,
    hop_length=160,
    n_jobs=n_jobs,
)
brain_segments['all'].shape, audio_segments.shape, layout.shape

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(torch.Size([199, 208, 400]),
 torch.Size([199, 128, 400]),
 torch.Size([208, 2]))

In [4]:
model = SimpleConv(SimpleConvConfig(transformer_layers=0)).to(device)


SimpleConv: 
	Params: 14432128
	Conv blocks: 5
	Trans layers: 0


In [5]:
output = model(
    {"meg": brain_segments["all"].to(device)},
    layout=layout.to(device),
    subjects=torch.full((brain_segments["all"].shape[0],), subject).to(device),
)

In [6]:
output.shape

torch.Size([199, 128, 400])